In [1]:
import timeit
import torch
import numpy as np

# import qpth.solvers.dynamic.solve as dynamic_solver
from qpth.util import get_sizes, extract_nBatch, expandParam, bdiag
from qpth.solvers import cvxpy, dynamic
from qpth.qp import QPFunction, QPSolvers

In [12]:
def get_kkt_problem(nBatch=2, nx=5, nineq=4, neq=3):
    def cast(m):
        # return m.cuda().double()
        return m.double()

    Q = cast(torch.randn(nx, nx))
    Q = Q.mm(Q.t())
    p = cast(torch.randn(nx))
    G = cast(torch.randn(nBatch, nineq, nx))
    h = cast(torch.randn(nBatch, nineq))
    A = cast(torch.randn(neq, nx))
    b = cast(torch.randn(neq))

    nBatch = extract_nBatch(Q, p, G, h, A, b)
    Q, _ = expandParam(Q, nBatch, 3)
    p, _ = expandParam(p, nBatch, 2)
    G, _ = expandParam(G, nBatch, 3)
    h, _ = expandParam(h, nBatch, 2)
    A, _ = expandParam(A, nBatch, 3)
    b, _ = expandParam(b, nBatch, 2)

    return Q, p, G, h, A, b

In [37]:
# Full Constraints

N_ITERS = 100

pdipm = QPFunction(verbose=-1, maxIter=20, solver=QPSolvers.PDIPM_BATCHED)
dynamic = QPFunction(verbose=-1, maxIter=20, solver=QPSolvers.DYNAMIC)
cvxpy = QPFunction(verbose=-1, maxIter=20, solver=QPSolvers.CVXPY)

num_same = 0
num_infeas = 0

time_p = 0
time_d = 0

showed = False

for _ in range(N_ITERS):
    Q, p, G, h, A, b = get_kkt_problem(1, 4, 3, 1)

    # Check for infeasibility with CVXPY
    try:
        zc = cvxpy(Q, p, G, h, A, b)
    except:
        num_infeas += 1
        continue

    start = timeit.default_timer()
    zp = pdipm(Q, p, G, h, A, b)

    mid = timeit.default_timer()
    zd = dynamic(Q, p, G, h, A, b)

    stop = timeit.default_timer()
    time_p += mid - start
    time_d += stop - start

    if np.allclose(zp, zd, rtol=1e-3):
        num_same += 1
        if not showed:
            showed = True
            print(Q, p, G, h)

print(num_same, num_infeas)
print(time_p, time_d)

tensor([[[ 3.6562, -0.9890,  1.8784, -1.8315],
         [-0.9890,  1.8881, -0.6961,  1.7295],
         [ 1.8784, -0.6961,  2.7366, -0.2663],
         [-1.8315,  1.7295, -0.2663,  2.2860]]], dtype=torch.float64) tensor([[ 2.8756, -0.0673,  0.0525,  1.2168]], dtype=torch.float64) tensor([[[ 3.1806,  0.1251, -0.7669,  0.2963],
         [ 2.3050,  0.5985, -0.2634,  0.3243],
         [ 1.6911, -0.9691, -0.2315,  0.8648]]], dtype=torch.float64) tensor([[ 0.0981, -0.4432,  0.9458]], dtype=torch.float64)
4 0
0.9806801119702868 1.0830009979981696


In [36]:
# Ineq Constraints

N_ITERS = 100

pdipm = QPFunction(verbose=-1, maxIter=20, solver=QPSolvers.PDIPM_BATCHED)
dynamic = QPFunction(verbose=-1, maxIter=50, solver=QPSolvers.DYNAMIC_INEQ)
cvxpy = QPFunction(verbose=-1, maxIter=20, solver=QPSolvers.CVXPY)

num_same = 0
num_infeas = 0

time_p = 0
time_d = 0

showed = False

for _ in range(N_ITERS):
    Q, p, G, h, A, b = get_kkt_problem(1, 3, 2, 0)

    # Check for infeasibility with CVXPY
    try:
        zc = cvxpy(Q, p, G, h, A, b)
    except:
        num_infeas += 1
        continue

    start = timeit.default_timer()
    zp = pdipm(Q, p, G, h, A, b)

    mid = timeit.default_timer()
    zd = dynamic(Q, p, G, h, A, b)

    stop = timeit.default_timer()
    time_p += mid - start
    time_d += stop - start

    if np.allclose(zp, zd, rtol=1e-2):
        num_same += 1
        if not showed:
            showed = True
            print(Q, p, G, h)

print(num_same, num_infeas)
print(time_p, time_d)

tensor([[[3.9246, 1.0582, 2.4789],
         [1.0582, 1.1527, 0.0584],
         [2.4789, 0.0584, 2.0558]]], dtype=torch.float64) tensor([[ 0.0156,  1.0281, -1.1223]], dtype=torch.float64) tensor([[[-0.9776, -0.2799, -1.0514],
         [ 1.1528, -2.9337, -0.3934]]], dtype=torch.float64) tensor([[-0.8359, -1.0936]], dtype=torch.float64)
45 0
0.771964763996948 0.8755008559601265
